In [1]:
import numpy as np
import pandas as pd

In [2]:
airbnb_df = pd.read_csv("../../data/raw/sf_airbnb_listings.csv")
airbnb_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,2.019060e+13,6/2/19,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,none,*Quiet cul de sac in friendly neighborhood *St...,...,t,f,moderate,f,f,1,1,0,0,1.60
1,5858,https://www.airbnb.com/rooms/5858,2.019060e+13,6/2/19,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,none,I love how our neighborhood feels quiet but is...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.90
2,7918,https://www.airbnb.com/rooms/7918,2.019060e+13,6/2/19,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,none,"Shopping old town, restaurants, McDonald, Whol...",...,f,f,strict_14_with_grace_period,f,f,9,0,9,0,0.14
3,8142,https://www.airbnb.com/rooms/8142,2.019060e+13,6/2/19,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...,Nice and good public transportation. 7 minute...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,9,0,9,0,0.14
4,8339,https://www.airbnb.com/rooms/8339,2.019060e+13,6/2/19,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,none,NaN,...,f,f,moderate,t,t,2,2,0,0,0.23


## Cleaning
We're going to want to get rid of any columns that we don't need. Because there are 106, this may be a bit of a challenge.

In [3]:
airbnb_df.drop(columns={"access", "bed_type", "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms", "calculated_host_listings_count", "calendar_last_scraped", "calendar_updated", "cancellation_policy", "city", "country_code", "country", "experiences_offered", "extra_people", "first_review", "guests_included", "has_availability", "host_about", "host_acceptance_rate", "host_has_profile_pic", "host_id", "host_identity_verified", "host_is_superhost", "host_location", "host_name", "host_neighbourhood", "host_picture_url", "host_response_rate", "host_response_time", "host_since", "host_thumbnail_url", "host_url", "host_verifications", "instant_bookable", "interaction", "is_business_travel_ready", "jurisdiction_names", "last_review", "last_scraped", "license", "listing_url", "market", "maximum_maximum_nights", "maximum_minimum_nights", "maximum_nights_avg_ntm", "medium_url", "minimum_maximum_nights", "minimum_minimum_nights", "minimum_nights_avg_ntm", "name", "neighborhood_overview", "notes", "number_of_reviews_ltm", "picture_url", "property_type", "require_guest_phone_verification", "require_guest_profile_picture", "requires_license", "review_scores_accuracy", "review_scores_checkin", "review_scores_cleanliness", "review_scores_communication", "scrape_id", "smart_location", "state", "street", "thumbnail_url", "transit", "xl_picture_url", "square_feet", "weekly_price", "monthly_price", "security_deposit", "cleaning_fee"}, inplace=True)
airbnb_df.keys()

Index(['id', 'summary', 'space', 'description', 'house_rules',
       'host_listings_count', 'host_total_listings_count', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'zipcode',
       'latitude', 'longitude', 'is_location_exact', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'review_scores_rating', 'review_scores_location',
       'review_scores_value', 'reviews_per_month'],
      dtype='object')

Check for duplicates:

In [4]:
airbnb_df["id"].value_counts().sort_values(ascending=False)[:5]

33474561    1
19642407    1
1056844     1
21026891    1
1620039     1
Name: id, dtype: int64

No duplicates, so we can remove this column:

In [5]:
airbnb_df = airbnb_df.drop(columns={"id"})

Determine the difference between `cleansed` columns and non-cleansed columns:

In [6]:
airbnb_df["neighbourhood_cleansed"].unique()

array(['Western Addition', 'Bernal Heights', 'Haight Ashbury', 'Mission',
       'Potrero Hill', 'Downtown/Civic Center', 'Castro/Upper Market',
       'Inner Sunset', 'South of Market', 'Noe Valley', 'Nob Hill',
       'Outer Richmond', 'Presidio Heights', 'Glen Park', 'Ocean View',
       'Pacific Heights', 'Financial District', 'Twin Peaks',
       'Russian Hill', 'Outer Sunset', 'Marina', 'Inner Richmond',
       'Excelsior', 'Seacliff', 'Chinatown', 'West of Twin Peaks',
       'Bayview', 'North Beach', 'Diamond Heights', 'Outer Mission',
       'Parkside', 'Lakeshore', 'Crocker Amazon', 'Golden Gate Park',
       'Visitacion Valley', 'Presidio'], dtype=object)

In [7]:
airbnb_df["neighbourhood"].unique()

array(['Lower Haight', 'Bernal Heights', 'Cole Valley', 'Alamo Square',
       'Western Addition/NOPA', 'Mission District', 'Potrero Hill',
       'Hayes Valley', 'Union Square', 'Haight-Ashbury', 'The Castro',
       'South Beach', 'Noe Valley', 'Nob Hill', 'Richmond District',
       'Presidio Heights', 'Glen Park', 'Dogpatch', 'Ingleside',
       'Pacific Heights', 'SoMa', 'Financial District', 'Twin Peaks',
       'Inner Sunset', 'Russian Hill', 'Outer Sunset', 'Marina',
       'Excelsior', 'Chinatown', 'Civic Center', 'Tenderloin',
       'Balboa Terrace', 'Bayview', 'Telegraph Hill', 'Diamond Heights',
       'Duboce Triangle', 'West Portal', 'North Beach', 'Mission Terrace',
       'Sunnyside', 'Cow Hollow', "Fisherman''''s Wharf", 'Lakeshore',
       "Fisherman''s Wharf", 'Crocker Amazon', 'Downtown', 'Oceanview',
       'Portola', 'Parkside', 'Presidio', 'Forest Hill',
       'Visitacion Valley', 'Mission Bay', 'Sea Cliff', 'Daly City',
       'Japantown'], dtype=object)

It looks like `neighbourhood_cleansed` is just a more specific version of the `neighbour_hood` column. We'll use the cleansed version:

In [8]:
airbnb_df.drop(columns={"neighbourhood"}, inplace=True)
airbnb_df = airbnb_df.rename(columns={"neighbourhood_cleansed": "neighborhood"})

In [9]:
airbnb_df["neighbourhood_group_cleansed"].unique()

array([nan])

Well this is useless

In [10]:
airbnb_df.drop(columns={"neighbourhood_group_cleansed"}, inplace=True)

Check for null values:

In [11]:
airbnb_df.isna().sum()

summary                       203
space                        1280
description                    24
house_rules                  2415
host_listings_count             0
host_total_listings_count       0
neighborhood                    0
zipcode                       245
latitude                        0
longitude                       0
is_location_exact               0
room_type                       0
accommodates                    0
bathrooms                      18
bedrooms                        2
beds                            7
amenities                       0
price                           0
minimum_nights                  0
maximum_nights                  0
availability_30                 0
availability_60                 0
availability_90                 0
availability_365                0
number_of_reviews               0
review_scores_rating         1537
review_scores_location       1542
review_scores_value          1542
reviews_per_month            1494
dtype: int64

Let's take this one step at a time:

In [12]:
# Set summary, space, description, house_rules to empty string
airbnb_df[["summary", "space", "description", "house_rules"]] = airbnb_df[["summary", "space", "description", "house_rules"]].fillna("")

# Delete any rows containing null values of bathrooms, bedrooms, and beds
indecesToDrop = list(airbnb_df[airbnb_df["bathrooms"].isna()].index)
indecesToDrop.extend(airbnb_df[airbnb_df["bedrooms"].isna()].index)
indecesToDrop.extend(airbnb_df[airbnb_df["beds"].isna()].index)
airbnb_df.drop(indecesToDrop, inplace=True)

airbnb_df.isna().sum()[airbnb_df.isna().sum() > 1]

zipcode                    245
review_scores_rating      1533
review_scores_location    1538
review_scores_value       1538
reviews_per_month         1491
dtype: int64

We will keep these as null values.

Next, let's fix the price. Because it is listed with a dollar sign, it is considered a string:

In [13]:
airbnb_df["price"][airbnb_df["price"].apply(lambda s: s[len(s)-3:] != "00 ")]

Series([], Name: price, dtype: object)

There are no prices with cents, so we'll just convert this into an int representing dollars:

In [14]:
airbnb_df["price"] = airbnb_df["price"].apply(lambda s: int(s[1:len(s)-4].replace(",", "")))
airbnb_df["price"].describe()

count    7549.000000
mean      215.617433
std       256.577478
min         0.000000
25%       100.000000
50%       152.000000
75%       248.000000
max      8000.000000
Name: price, dtype: float64

What?? Why do we have a 0? Let's drop it.

In [15]:
indeces_to_drop = airbnb_df[airbnb_df["price"] == 0].index
airbnb_df.drop(indeces_to_drop, inplace=True)
airbnb_df["price"].describe()

count    7548.000000
mean      215.645999
std       256.582470
min        10.000000
25%       100.000000
50%       152.000000
75%       248.250000
max      8000.000000
Name: price, dtype: float64

In [16]:
airbnb_df[airbnb_df["summary"].apply(lambda s: "'" in s)].iloc[0][0][180:280]

"ur 100 year old home.  We''''ve been Hosts since Airbnb''''s 1st year (2009), and can suggest restau"

We have a problem with single quotes showing up multiple times in strings. Let's find all string values and fix this single quote problem:

In [17]:
types = (airbnb_df.dtypes == object)
col_w_str = types[types].index
indeces_to_delete = list(np.argwhere(col_w_str == "is_location_exact"))
indeces_to_delete.extend(np.argwhere(col_w_str == "zipcode"))
col_w_str = np.delete(col_w_str, indeces_to_delete)

In [18]:
import re

airbnb_df[col_w_str] = airbnb_df[col_w_str].apply(lambda row: row.apply(lambda s: "'".join(re.split(r"'+", s))), axis=1)

Let's also add the supervisor district:

In [19]:
import os
import sys
module_path = os.path.abspath(os.path.join('..')).split("/")
module_path = "/".join(module_path[:len(module_path) - 1]) # We have to go one level up
if module_path not in sys.path:
    sys.path.append(module_path)
from src.features.supervisor_districts import sd
sd.get_district(airbnb_df["longitude"].iloc[0], airbnb_df["latitude"].iloc[0])

{'district_num': 8, 'name': 'Mandelman'}

In [20]:
def get_district(row):
    try:
        return sd.get_district(row["longitude"], row["latitude"])["district_num"]
    except:
        return np.nan

airbnb_df["supervisor_district"] = airbnb_df.apply(get_district, axis=1)
airbnb_df["supervisor_district"].describe()

count    7542.000000
mean        6.022673
std         2.838095
min         1.000000
25%         3.000000
50%         6.000000
75%         8.000000
max        11.000000
Name: supervisor_district, dtype: float64

In [21]:
airbnb_df["supervisor_district"].value_counts()

8.0     1261
3.0     1122
5.0      918
9.0      875
6.0      807
2.0      574
10.0     508
4.0      391
7.0      384
1.0      354
11.0     348
Name: supervisor_district, dtype: int64

No `NaN`s!

Looks good to me! Let's export:

In [22]:
airbnb_df.to_csv("../../data/processed/sf_airbnb_listings_clean.csv")